In [1]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt

In [2]:
Data_root_dir="/content/drive/MyDrive/Dataset/FashionMnist/Data"

In [3]:
train_data=torchvision.datasets.FashionMNIST(root=Data_root_dir,
                                             train=True,
                                             download=True,
                                             transform=torchvision.transforms.ToTensor())


test_data=torchvision.datasets.FashionMNIST(root=Data_root_dir,
                                            train=False,
                                            download=True,
                                            transform=torchvision.transforms.ToTensor())

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Dataset/FashionMnist/Data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /content/drive/MyDrive/Dataset/FashionMnist/Data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Dataset/FashionMnist/Data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/drive/MyDrive/Dataset/FashionMnist/Data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Dataset/FashionMnist/Data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/drive/MyDrive/Dataset/FashionMnist/Data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Dataset/FashionMnist/Data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/drive/MyDrive/Dataset/FashionMnist/Data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
Batch_size=64
train_data_loader=torch.utils.data.DataLoader(train_data,batch_size=Batch_size)
test_data_loader=torch.utils.data.DataLoader(test_data,batch_size=Batch_size)


In [5]:
print(type(train_data_loader))
print(type(test_data_loader))

<class 'torch.utils.data.dataloader.DataLoader'>
<class 'torch.utils.data.dataloader.DataLoader'>


In [6]:
for X,y in test_data_loader:
  print(X.shape)
  print(y.shape,y.dtype,torch.unique(y))
  break

torch.Size([64, 1, 28, 28])
torch.Size([64]) torch.int64 tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [7]:
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [8]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork,self).__init__()

    self.flatten=nn.Flatten()

    self.linear_relu_stack=nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,300),
        nn.ReLU(),
        nn.Linear(300,100),
        nn.ReLU(),
        nn.Linear(100,50),
        nn.ReLU(),
        nn.Linear(50,10),
        nn.ReLU()
    )
  def forward(self,X):
    X=self.flatten(X)
    logits=self.linear_relu_stack(X)
    return logits

In [9]:
model=NeuralNetwork().to(device)

In [10]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=300, bias=True)
    (3): ReLU()
    (4): Linear(in_features=300, out_features=100, bias=True)
    (5): ReLU()
    (6): Linear(in_features=100, out_features=50, bias=True)
    (7): ReLU()
    (8): Linear(in_features=50, out_features=10, bias=True)
    (9): ReLU()
  )
)


In [11]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=1e-3)

In [12]:
def train(data_loader,model,loss_fn,optimizer):
  size=len(data_loader.dataset)

  for batch, (X,y) in enumerate(data_loader):
    X,y =X.to(device),y.to(device)
    
    pred=model(X)
    loss=loss_fn(pred,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch%100==0:
      #loss=current=loss.item(),batch*len(X)
      loss, current = loss.item(), batch * len(X)
      print(f"training loss: {loss} [{current}/{size}]")

In [13]:
def test(data_loader,model):
  size=len(data_loader.dataset)
  model.eval()

  test_loss,correct=0,0

  with torch.no_grad():
    for X,y in data_loader:
      X,y =X.to(device),y.to(device)
      pred=model(X)
      test_loss=test_loss+loss_fn(pred,y).item()
      correct=correct+(pred.argmax(1)==y).type(torch.float).sum().item()
  test_loss=test_loss/size
  correct=correct/size

  print(f"Correct Predictions: {100*correct}%, Average Loss: {test_loss*100}%")

In [14]:
Epoch=500

for epoch in range(Epoch):
  print(f"\n\n\n\nEpoch: {epoch+1}")

  train(train_data_loader,model,loss_fn,optimizer)
  test(test_data_loader,model)

print("End of training")

Streaming output truncated to the last 5000 lines.
training loss: 0.715046763420105 [25600/60000]
training loss: 1.250493049621582 [32000/60000]
training loss: 0.9276719689369202 [38400/60000]
training loss: 1.1882702112197876 [44800/60000]
training loss: 1.1273798942565918 [51200/60000]
training loss: 1.0693398714065552 [57600/60000]
Correct Predictions: 57.550000000000004%, Average Loss: 1.692994436621666%




Epoch: 189
training loss: 1.1687965393066406 [0/60000]
training loss: 1.0888267755508423 [6400/60000]
training loss: 1.1514068841934204 [12800/60000]
training loss: 1.0700771808624268 [19200/60000]
training loss: 0.7147770524024963 [25600/60000]
training loss: 1.2504658699035645 [32000/60000]
training loss: 0.9276928305625916 [38400/60000]
training loss: 1.1875864267349243 [44800/60000]
training loss: 1.1268346309661865 [51200/60000]
training loss: 1.0689204931259155 [57600/60000]
Correct Predictions: 57.56%, Average Loss: 1.6925915986299513%




Epoch: 190
training loss: 1.168

In [15]:
#torch.save(model, "model.pth")

In [16]:
torch.save(model, "/content/drive/MyDrive/Dataset/FashionMnist/Fashion_Mnist_Pytorch_model.pth")

In [17]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

device1="cpu"
model.to(device1)
model.eval()
test_data[0][0]
X, y = torch.tensor(test_data[0][0]).to(device1), torch.tensor(test_data[0][1]).to(device1)

with torch.no_grad():
  pred = model(X)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(predicted, actual)

Ankle boot Ankle boot


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
